# KNIT7
# Query Elasticsearch and Upload Data to Bigquery

## Import 

In [ ]:
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from mflib.mflib import MFLib
try:
    fablib = fablib_manager()                
    #fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Slice info

In [ ]:
# Change your slice name
slice_name = "MyMonitoredSlice"

In [ ]:
%%time
mf = MFLib(slice_name)

In [ ]:
container_name= 'elk-bigquery'

## Find Measurement Node

In [ ]:
meas_node_name = mf.measurement_node_name
try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)   

try:
    meas_node = slice.get_node(name=meas_node_name)
except Exception as e:
    print(f"Fail: {e}")   
print (meas_node)

## Find Elasticsearch Index Name

In [ ]:
meas_node.execute("curl http://localhost:9200/_aliases?pretty=true", quiet=True)

## Setup Tunnel and View Kibana UI

In [ ]:
# The ELK service was created by the mf.instrumentize call.
# Get access info for Kibana by using the mflib.info call to the elk service.
# Create a dictionary to pass to the service.
data = {}
# Set the info you want to get.
data["get"] = ["nginx_id", "nginx_password"]
# Call info using service name and data dictionary.
info_results = mf.info("elk", data)
print(info_results)

if info_results["success"]:
    print(f"user: {info_results['nginx_id']} \npass: {info_results['nginx_password']}")

# ELK SSH Tunnel Command
# mf.kibana_tunnel_local_port = 10020 # optionally change the port
print(mf.kibana_tunnel)

print(f"Browse to http://localhost:{mf.kibana_tunnel_local_port}/")

## Specify Google Project/Dataset/Table Name

In [ ]:
# Change project/dataset name to the ones you just created. 
project_name = 'elk-bigquery'
packetbeat_dataset = 'packetbeat'
filebeat_dataset = 'filebeat'
metricbeat_dataset = 'metricbeat'
# Table name can be random
table = 'test'

## Filebeat data

In [ ]:
# Specify the elasticsearch index name 
index = 'filebeat-7.13.2-2023.09.12-000001'
query = '{"query":{"range":{"@timestamp":{"gte":"now-10m"}}}}'
# Bigquery table name: project.dataset.tablename
table_name = f'{project_name}.{filebeat_dataset}.{table}'
key_string = f'--key key.json'
code_file_name = 'elk-bigquery.py'
command = f'''sudo docker exec -i {container_name} python3 {code_file_name} --query '{query}' --index {index} {key_string} --table {table_name }'''
print (command)

In [ ]:
meas_node.execute(command)

## Packetbeat Data

In [ ]:
# Specify the elasticsearch index name
index = 'packetbeat-7.13.2-2023.09.12-000001'
query = '{"query":{"range":{"@timestamp":{"gte":"now-10m"}}}}'
# Bigquery table name: project.dataset.tablename
table_name = f'{project_name}.{packetbeat_dataset}.{table}'
key_string = f'--key key.json'
code_file_name = 'elk-bigquery.py'
command = f'''sudo docker exec -i {container_name} python3 {code_file_name} --query '{query}' --index {index} {key_string} --table {table_name }'''
print (command)

In [ ]:
meas_node.execute(command)

## Metricbeat Data

In [ ]:
# Specify the elasticsearch index name
index = 'metricbeat-7.13.2-2023.09.12-000001'
query = '{"query":{"range":{"@timestamp":{"gte":"now-10m"}}}}'
# Bigquery table name: project.dataset.tablename
table_name = f'{project_name}.{metricbeat_dataset}.{table}'
key_string = f'--key key.json'
code_file_name = 'elk-bigquery.py'
command = f'''sudo docker exec -i {container_name} python3 {code_file_name} --query '{query}' --index {index} {key_string} --table {table_name }'''
print (command)

In [ ]:
meas_node.execute(command)

## Switch to Bigquery web UI to view the data

https://cloud.google.com/bigquery